In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
#from pyemma import msm # not available on Kaggle Kernel
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM


In [ ]:
import datetime as dt
import os

In [ ]:
pd.__version__

In [ ]:
# 1x_uN2NhXl2-o9YF3J0GyLcxxNGkPXU9h

In [ ]:
if not os.path.exists('ms_proj'):
  os.makedirs('ms_proj')
%cd ms_proj
if not os.path.exists('data'):
  os.makedirs('data')
%cd data

In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

In [ ]:
file_id = r'1x_uN2NhXl2-o9YF3J0GyLcxxNGkPXU9h'
destination = r'cleaned_data.csv'


In [ ]:
download_file_from_google_drive(file_id, destination)

In [ ]:
# Set some parameters to get good visuals - style to ggplot and size to 15,10
plt.style.use('ggplot')
import matplotlib.style as style
style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 8)

In [ ]:
# return Series of distance between each point and his distance with the closest centroid
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.set_value(i, np.linalg.norm(Xa-Xb))
    return distance

In [ ]:
df = pd.read_csv('cleaned_data.csv')

In [ ]:
df['@timestamp'] = pd.to_datetime(df['@timestamp'])

In [ ]:
df['@timestamp.1'] = pd.to_datetime(df['@timestamp.1'])

In [ ]:
df.loc[df['@timestamp.1'] != df['@timestamp']]

In [ ]:
df.head()

In [ ]:
df['date'] = df['@timestamp'].dt.date


In [ ]:
df['hour'] = df['@timestamp'].dt.hour

In [ ]:
df['minute'] = df['@timestamp'].dt.minute

In [ ]:
df_counts = df.groupby(['date', 'hour', 'minute'])['_id'].count().reset_index()

In [ ]:
df_counts.iloc[0]['date'].strftime('%Y-%m-%d')+" "+str(df_counts.iloc[0]['hour'])+":"+str(df_counts.iloc[0]['minute'])+":00"

In [ ]:
df_counts['date_trunc'] = pd.to_datetime(df_counts.apply(lambda row: row['date'].strftime('%Y-%m-%d')+" "+str(row['hour'])+":"+str((row['minute']//10)*10)+":00", axis=1))

In [ ]:
df_counts

In [ ]:
df_counts_final = df_counts.groupby('date_trunc')['_id'].sum().reset_index()

In [ ]:
df_counts_final.plot(x='date_trunc', y='_id')

In [ ]:
df_counts_final.sort_values('_id', ascending=False).head()

In [ ]:
df_counts_final = df_counts_final.loc[df_counts_final['_id']<1000]

In [ ]:
df = df_counts_final.copy()

In [ ]:
df = df.rename(columns={'_id':'count'})

In [ ]:
df['hours'] = df['date_trunc'].dt.hour
df['minutes'] = (df['date_trunc'].dt.minute//10 < 3).astype(int)
df['daylight'] = ((df['hours'] >= 6) & (df['hours'] <= 22)).astype(int)

In [ ]:
df.head(10)

In [ ]:
# the day of the week (Monday=0, Sunday=6) and if it's a week end day or week day.
df['DayOfTheWeek'] = df['date_trunc'].dt.dayofweek
# df['WeekDay'] = (df['DayOfTheWeek'] < 5).astype(int)


In [ ]:
df['time_epoch'] = (df['date_trunc'].astype(np.int64)/100000000000).astype(np.int64)

In [ ]:
# df['categories'] = df['WeekDay']*2 + df['daylight']

In [ ]:
data = df[['count', 'hours', 'minutes']] #'DayOfTheWeek',, 'WeekDay', , 'minutes'
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# reduce to 2 importants features
pca = PCA(n_components=2, random_state=123)
data = pca.fit_transform(data)
# standardize these 2 new features
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)

In [ ]:
n_cluster = range(1, 20)
kmeans = [KMeans(n_clusters=i, random_state=123).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]
fig, ax = plt.subplots()
ax.plot(n_cluster, scores)
plt.show()

In [ ]:
cluster_model = kmeans[14]

In [ ]:
df['cluster'] = cluster_model.predict(data)
df['principal_feature1'] = data[0]
df['principal_feature2'] = data[1]
df['cluster'].value_counts()

In [ ]:
fig, ax = plt.subplots()
colors = {0:'red', 1:'blue', 2:'green', 3:'pink', 4:'black', 5:'orange', 6:'cyan', 7:'yellow', 8:'brown', 9:'purple', 10:'white', 
          11: 'grey', 12:'lightblue', 13:'lightgreen', 14: 'darkgrey'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["cluster"].apply(lambda x: colors[x]))
ax.scatter(cluster_model.cluster_centers_[:,0], cluster_model.cluster_centers_[:,1], s=100, c='darkblue')
plt.show()

In [ ]:
outliers_fraction = 0.003

In [ ]:
distance = getDistanceByPoint(data, cluster_model)
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
# anomaly21 contain the anomaly result of method 2.1 Cluster (0:normal, 1:anomaly) 
df['anomaly21'] = (distance >= threshold).astype(int)

In [ ]:
df

In [ ]:
df = df.dropna()

In [ ]:
# visualisation of anomaly with cluster view
fig, ax = plt.subplots()
colors = {0:'blue', 1:'red'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["anomaly21"].apply(lambda x: colors[x]))
plt.show()

In [ ]:
fig, ax = plt.subplots()

a = df.loc[df['anomaly21'] == 1, ['time_epoch', 'count']] #anomaly
# b = df.loc[df['count'] >30000, ['time_epoch', 'count']] #missed

ax.plot(df['time_epoch'], df['count'], color='blue')
ax.scatter(a['time_epoch'],a['count'], color='red', s=100)
# ax.scatter(b['time_epoch'],b['count'], color='red', s=100)
plt.show()

In [ ]:
a = df.loc[df['anomaly21'] == 0, 'count']
b = df.loc[df['anomaly21'] == 1, 'count']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
plt.legend()
plt.show()


## Isolated forests

In [ ]:
outliers_fraction = 0.005

In [ ]:
# Take useful feature and standardize them 
data = df[['count', 'hours', 'daylight']] # 'DayOfTheWeek', 'WeekDay'
# data = df[['principal_feature1', 'principal_feature2']]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# train isolation forest 
model =  IsolationForest(contamination = outliers_fraction)
model.fit(data)
# add the data to the main  
df['anomaly25'] = pd.Series(model.predict(data))
df['anomaly25'] = df['anomaly25'].map( {1: 0, -1: 1} )
print(df['anomaly25'].value_counts())

In [ ]:
# visualisation of anomaly throughout time (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly25'] == 1, ['time_epoch', 'count']] #anomaly
b = df.loc[df['anomaly21']==1, ['time_epoch', 'count']]

ax.plot(df['time_epoch'], df['count'], color='blue')
ax.scatter(b['time_epoch'],b['count'], color='green', s=500)
ax.scatter(a['time_epoch'],a['count'], color='red', s=100)

plt.show()

In [ ]:
# visualisation of anomaly with cluster view
fig, ax = plt.subplots()
colors = {0:'blue', 1:'red'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["anomaly21"].apply(lambda x: colors[x]))
plt.show()

In [ ]:
a = df.loc[df['anomaly25'] == 0, 'count']
b = df.loc[df['anomaly25'] == 1, 'count']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label = ['normal', 'anomaly'])
plt.legend()
plt.show()

## SVM

In [ ]:
# Take useful feature and standardize them 
data = df[['count', 'hours', 'daylight', 'DayOfTheWeek']]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)

In [ ]:
# train one class SVM 
model =  OneClassSVM(nu=0.95 * outliers_fraction) #nu=0.95 * outliers_fraction  + 0.05
data = pd.DataFrame(np_scaled)
model.fit(data)
# add the data to the main  
df['anomaly26'] = pd.Series(model.predict(data))
df['anomaly26'] = df['anomaly26'].map( {1: 0, -1: 1} )
print(df['anomaly26'].value_counts())

In [ ]:
# visualisation of anomaly throughout time (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly26'] == 1, ['time_epoch', 'count']] #anomaly

ax.plot(df['time_epoch'], df['count'], color='blue')
ax.scatter(a['time_epoch'],a['count'], color='red', s=100)
plt.show()

In [ ]:
# visualisation of anomaly with cluster view
fig, ax = plt.subplots()
colors = {0:'blue', 1:'red'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["anomaly26"].apply(lambda x: colors[x]))
plt.show()

## RNNs

In [ ]:
#select and standardize data
data_n = df[['count', 'hours', 'minutes']]#'daylight', 'DayOfTheWeek', 'WeekDay'
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data_n)
data_n = pd.DataFrame(np_scaled)

In [ ]:
# important parameters and train/test size
prediction_time = 1 
testdatasize = 50
unroll_length = 10
testdatacut = testdatasize + unroll_length  + 1

#train data
x_train = data_n[0:-prediction_time-testdatacut].as_matrix()
y_train = data_n[prediction_time:-testdatacut  ][0].as_matrix()

# test data
x_test = data_n[0-testdatacut:-prediction_time].as_matrix()
y_test = data_n[prediction_time-testdatacut:  ][0].as_matrix()

In [ ]:
#unroll: create sequence of 50 previous data points for each data points
def unroll(data,sequence_length=24):
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)

In [ ]:
# adapt the datasets for the sequence data shape
x_train = unroll(x_train,unroll_length)
x_test  = unroll(x_test,unroll_length)
y_train = y_train[-x_train.shape[0]:]
y_test  = y_test[-x_test.shape[0]:]

# see the shape
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

In [ ]:
# specific libraries for RNN
# keras is a high layer build on Tensorflow layer to stay in high level/easy implementation
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time #helper libraries
from keras.models import model_from_json
import sys

In [ ]:
# Build the model
model = Sequential()

model.add(LSTM(
    input_dim=x_train.shape[-1],
    output_dim=10,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    200,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    units=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print('compilation time : {}'.format(time.time() - start))

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=1000,
    nb_epoch=30,
    validation_split=0.1)

In [ ]:
diff=[]
ratio=[]
p = model.predict(x_test)

In [ ]:
for u in range(len(y_test)):
    pr = p[u][0]
    ratio.append((y_test[u]/pr)-1)
    diff.append(abs(y_test[u]- pr))

In [ ]:
fig, axs = plt.subplots()
axs.plot(p,color='red', label='prediction')
axs.plot(y_test,color='blue', label='y_test')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# select the most distant prediction/reality data points as anomalies
diff = pd.Series(diff)
number_of_outliers = int(0.1*len(diff))
threshold = diff.nlargest(number_of_outliers).min()

In [ ]:
# data with anomaly label (test data part)
test = (diff >= threshold).astype(int)

In [ ]:
test.head()

In [ ]:
# the training data part where we didn't predict anything (overfitting possible): no anomaly
complement = pd.Series(0, index=np.arange(len(data_n)-testdatasize))

In [ ]:
complement.shape

In [ ]:
# # add the data to the main
df['anomaly27'] = complement.append(test, ignore_index='True')
print(df['anomaly27'].value_counts())

In [ ]:
# visualisation of anomaly throughout time (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly27'] == 1, ['time_epoch', 'count']] #anomaly

ax.plot(df['time_epoch'], df['count'], color='blue')
ax.scatter(a['time_epoch'],a['count'], color='red', s=100)
# plt.axis([1.370*1e7, 1.405*1e7, -2.5,1])
plt.show()

In [ ]:
diff=[]
ratio=[]
p = model.predict(x_train)

In [ ]:
for u in range(len(y_train)):
    pr = p[u][0]
    ratio.append((y_train[u]/pr)-1)
    diff.append(abs(y_train[u]- pr))

In [ ]:
fig, axs = plt.subplots()
axs.plot(p,color='red', label='prediction')
axs.plot(y_train,color='blue', label='y_test')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# select the most distant prediction/reality data points as anomalies
diff = pd.Series(diff)
number_of_outliers = int(0.01*len(diff))
threshold = diff.nlargest(number_of_outliers).min()

In [ ]:
diff.shape

In [ ]:
diff.index

In [ ]:
complement.head()

In [ ]:
# data with anomaly label (test data part)
train = (diff >= threshold).astype(int)
# the training data part where we didn't predict anything (overfitting possible): no anomaly
# complement = pd.Series(0, index=np.arange(len(diff)))
# # add the data to the main
df['anomaly28'] = train.append(complement, ignore_index='True')
print(df['anomaly28'].value_counts())

# visualisation of anomaly throughout time (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly28'] == 1, ['time_epoch', 'count']] #anomaly

ax.plot(df['time_epoch'], df['count'], color='blue')
ax.scatter(a['time_epoch'],a['count'], color='red', s= 100)
# plt.axis([1.370*1e7, 1.405*1e7, -2.5,1])
plt.show()